In [43]:
!pip install natasha
!pip install pymorphy2
import json
import pandas as pd
import re
import pymorphy2
from natasha import NewsNERTagger
from natasha import MorphVocab, NewsEmbedding, NewsMorphTagger
from natasha import Doc, Segmenter
analyzer = pymorphy2.MorphAnalyzer()
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn import metrics
from sklearn.cluster import KMeans

In [4]:
embedding = NewsEmbedding()
segmenter = Segmenter()

In [5]:
morph_tagger = NewsMorphTagger(embedding)
morph_vocab = MorphVocab()
ner_tagger = NewsNERTagger(embedding)

In [6]:
with open("new_corpus.json") as f:
    docs = json.load(f)

In [77]:
titles = list(docs.keys())
texts = list(docs.values())
df = pd.DataFrame.from_dict({'doc_num':titles, 'text':texts})
df.set_index('doc_num', inplace=True)

In [79]:
df.head()

,text
doc_num,
0,Северодвинск — город в Архангельской области ...
1,Абхазия — регион в северо-западной части южно...
2,Авария на Чернобыльской АЭС — разрушение 26 а...
3,Авнюгский — посёлок в Верхнетоемском районе А...
4,«Авторадио» — российская музыкальная радиоста...


In [71]:
def NER (transcript):
  script = Doc(re.sub(r'\((.*?)\)', "", transcript))
  script.segment(segmenter)
  script.tag_morph(morph_tagger)
  for token in script.tokens:
    token.lemmatize(morph_vocab)
  script.tag_ner(ner_tagger)
  for span in script.spans:
    span.normalize(morph_vocab)
  named_ents = [(i.text, i.type, i.normal) for i in script.spans]
  normed_ents = []
  for word, tag, norm in named_ents:
    if len(word.split()) == 1 and tag == "LOC":
      for gram in range(len(analyzer.parse(word))):
        if "Geox" in analyzer.parse(word)[gram].tag:
          normed_ents.append((analyzer.parse(word)[gram].normal_form))
          break
        elif gram == len(analyzer.parse(word)) - 1:
          normed_ents.append((norm.lower().strip(".,!?;-")))
    else:
      normed_ents.append((norm.lower().strip(".,!?;-")))
  return sorted(normed_ents)

In [72]:
df["named_entities"] = df.apply(lambda row: NER(row["text"]), axis=1)

In [73]:
df.head()

,text,named_entities
title,,
0,Северодвинск — город в Архангельской области ...,"[архангельская область, город, крайний север, ..."
1,Абхазия — регион в северо-западной части южно...,"[абжуа, абхазия, абхазская автономная республи..."
2,Авария на Чернобыльской АЭС — разрушение 26 а...,"[аэс, белоруссия, европа, нагасаки, припять, р..."
3,Авнюгский — посёлок в Верхнетоемском районе А...,"[авнюгский, архангельская область, верхнетоемс..."
4,«Авторадио» — российская музыкальная радиоста...,"[авторадио, авторадио, авторадио, авторадио, а..."


In [74]:
has_ner = [i for i in df.index.values if df.named_entities[i]]

In [40]:
len(has_ner), df.shape[0] # выведет количество док-в с именованными сущностями и общее кол-во док-в

(163, 201)

In [75]:
df_ner = df[df.index.isin(has_ner)]

In [44]:
ner_voc = []
for row in df_ner.named_entities.tolist():
  ner_voc.extend(row)
len(set(ner_voc)), len(ner_voc) # выведем кол-во уникальных сущностей и общее количество сущностей

(983, 1979)

In [48]:
vocabulary = sorted(set(ner_voc))
corpus = df_ner.named_entities.apply(str).tolist()

In [60]:
pipe = Pipeline([('count', CountVectorizer(vocabulary = vocabulary)),
                 ('tfid', TfidfTransformer())]).fit(corpus)
X = pipe.fit_transform(corpus)
km = KMeans(n_clusters = 30, init = 'k-means++', max_iter = 600, 
            algorithm = "full", precompute_distances = True)

In [61]:
km.fit(X)

KMeans(algorithm='full', copy_x=True, init='k-means++', max_iter=600,
       n_clusters=30, n_init=10, n_jobs=None, precompute_distances=True,
       random_state=None, tol=0.0001, verbose=0)

Для оценки качества проведенной кластеризации используем 2 метрики:

1.   Индекс Дэвиcа-Болдуина: оценивает расстояние от объекта кластера до центроида и расстояние между центроидами; чем он ниже, тем лучше разбиение. В нашем случае разбиение прошло успешно: с увеличением числа кластеров он уменьшается
2.   Силуэт: показывает, насколько объект похож на свой кластер относительно других кластеров; если значение стремится к 1 - хорошее разбиение, если к -1 - плохое, если в районе 0 - кластеры пересекаются. Наш результат не намного больше 0, поэтому кластеры во многом схожи


In [62]:
print('Индекс Дэвиса-Болдуина: ', metrics.davies_bouldin_score(X.toarray(), km.labels_))
print('Силуэт: ', metrics.silhouette_score(X, km.labels_, sample_size=1000))

Индекс Дэвиса-Болдуина:  1.4669628222326678
Силуэт:  0.1743023639029352


In [63]:
df_ner["label"] = km.predict(X)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [64]:
df_ner["label"].value_counts()

4     79
6      7
13     6
1      6
2      5
29     5
7      4
18     4
11     4
25     3
22     3
20     3
8      3
10     3
28     3
5      2
12     2
14     2
15     2
16     2
17     2
19     2
21     2
23     2
26     2
9      1
24     1
3      1
27     1
0      1
Name: label, dtype: int64

In [66]:
df_ner.query("label == 20")

,text,named_entities,label
title,,,
98,"Сумерки — интервал времени, в течение которого...","[земля, солнце, солнце, солнце]",20
118,Зимнее солнцестояние — один из двух дней в год...,"[солнце, солнце, солнце]",20
172,"Летнее солнцестояние — момент, когда Солнце в ...",[солнце],20


In [69]:
df_ner.query("label == 10")

,text,named_entities,label
title,,,
87,"Город — крупный населённый пункт, жители котор...","[город, древняя русь]",10
173,"Русские летописи — летописи, исторические сочи...","[великий княжестве литовский, киев, киевская р...",10
191,"Монгольское нашествие на Русь, также известное...","[батый, батый, западный поход, монгольская имп...",10


In [67]:
df_ner.query("label == 2")

,text,named_entities,label
title,,,
55,Верхнее Шилово — деревня в Красноборском район...,"[архангельская область, верхнее шилово, красно...",2
90,Городищенская — сельский населённый пункт в Кр...,"[архангельская область, городищенская, красноб...",2
109,Дябрино — посёлок в Красноборском районе Архан...,"[алексеевское сельское поселение, архангельска...",2
158,Красноборский район — административно-территор...,"[архангельская область, красноборск, краснобор...",2
190,Монастырская Пашня — это сельский населённый п...,"[архангельская область, красноборский район, м...",2


In [68]:
order_centroids = km.cluster_centers_.argsort()[:, ::-1]
terms = pipe[0].get_feature_names()
for i in range(30):
  print("Cluster %d:" % i, end='')
  for ind in order_centroids[i, :10]:
    print(' %s' % terms[ind], end='')
  print()

Cluster 0: мурманск россия кольский запад баренцево город север западный поход западная европа западная сибирь
Cluster 1: земля евразия европа виктория архангельская россия испания колгуев франция ненецкий
Cluster 2: красноборский архангельская дябрино городищенская красноборск уфтюга северная западный поход закавказье запад
Cluster 3: ссср цик президиум ярославль закавказье запад западная европа западная сибирь западный поход заполярный район
Cluster 4: россия архангельская архангельск ссср северная белоруссия виноградовский рф европа украина
Cluster 5: архангельский архангельск вологодский архангельская россия ссузы архангельское запад заполярный район западная европа
Cluster 6: северодвинск архангельская звездочка кудемскай россия город север рф зато западная европа
Cluster 7: рсфср ссср кпсс ран рф советский россия ярославль западная европа закавказье
Cluster 8: вмф ссср ораниенбаум приозерск адмиралтейства россия кронштадт заполярье заполярный транссиб заполярный район
Cluster 9: 

### Вывод
Кластеризация прошла успешно, но мы можем наблюдать пересечение содержания некоторых кластеров (например, 3 и 29), что говорит лишь об очень смежных темах в собранных документах (север России, исторические события, природа и т.д.)